## FOCUSED
- Author:  Reshama Shaikh

## Data Sources
1. S&P 500
1. OSCI Index
1. Suzy spreadsheet
1. OSPO Landscape (Todo group)

---

## Date

In [1]:
from datetime import date

current_date = date.today()
print("Today's date:", current_date)

Today's date: 2022-05-21


## Import libraries


In [2]:
import pandas as pd 
import numpy as np
import requests
import time
import csv
from pprint import pprint

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
%load_ext watermark
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.25.0

Compiler    : Clang 11.1.0 
OS          : Darwin
Release     : 19.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Git hash: 4b28b648a682462bee305c284f3bdd5d5d41c43a

csv     : 1.0
plotly  : 5.4.0
numpy   : 1.20.2
requests: 2.25.1
pandas  : 1.3.3



## Set pandas view options

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1. Data Source:  S&P 500

In [6]:
datapath="data_raw/"

def get_raw_data(data_url, file_name):
    file_url = data_url + "/" + file_name
    !wget -N {file_url} -P {datapath} 
    
def read_raw_data(file_name):
    df_use = pd.read_csv(datapath+file_name)
    
    return df_use

In [7]:
data_url='https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/'
file_name = 'constituents.csv'

#run once
#get_raw_data(data_url, file_name)

In [8]:
# run once
#!mv data_raw/constituents.csv data_raw/sp500.csv 

In [9]:
file_name='sp500.csv'
df_sp = read_raw_data(file_name)

print(df_sp.shape)
df_sp.head(5)

(505, 3)


,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


### 2. Data Source:  OSCI

In [10]:
data_url='https://ststaticprodosciwebz2vmu.blob.core.windows.net/data/osci-ranking/daily'
file_name = '2022-04-17.json'

#run once
#get_raw_data(data_url, file_name)

In [11]:
# run once
#!mv data_raw/2022-04-17.json data_raw/osci.json 

In [12]:
file_name='osci.json'
file_name

'osci.json'

In [13]:
import json
 
# Opening JSON file
f = open('data_raw/osci.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

print(len(data))
#print(data)


# Iterating through the json
# list
for i in data['date'][0]:
    print(i)
 
# Closing file
f.close()

3
2


In [14]:
print(data['date'])
print(type(data))
print(type(data['date']))
#print(data['date'].['comparedDate'])
#print(data['date'][0]['data'])
print(data['date'], data['data'][0]['company'])

2022-04-17
<class 'dict'>
<class 'str'>
2022-04-17 Google


In [15]:
#pd.read_json('data_raw/osci.json')

In [16]:
# Normalizing data
multiple_level_data = pd.json_normalize(data, record_path =['data'])
#, meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], 
#meta_prefix='config_params_', record_prefix='dbscan_')

# Saving to CSV format
#multiple_level_data.to_csv('multiplelevel_normalized_data.csv', index=False)

multiple_level_data.head(1)

,positionChange,company,activeContributors,activeContributorsChange,totalCommunity,totalCommunityChange,position,yoy,contributors,languages,licenses,industry
0,0.0,Google,4017.0,273.0,8676,431.0,1,"[{'date': '2020-04-17', 'active': 2587.0}, {'d...","[{'Contributor': 'Modular Magician', 'Commits'...","[{'name': 'Brightscript', 'amount': 5}, {'name...","[{'name': 'apache-2.0', 'amount': 180027}, {'n...",Technology


In [17]:
keep_cols = ['positionChange', 'company', 'activeContributors', 'activeContributorsChange', 
             'totalCommunity', 'totalCommunityChange', 'position', 'industry'
            ]

df_osci = multiple_level_data.filter(keep_cols)
df_osci['in_osci']=1

print(df_osci.shape)
df_osci.head(5)

(298, 9)


,positionChange,company,activeContributors,activeContributorsChange,totalCommunity,totalCommunityChange,position,industry,in_osci
0,0.0,Google,4017.0,273.0,8676,431.0,1,Technology,1
1,0.0,Microsoft,3630.0,308.0,8128,545.0,2,Technology,1
2,0.0,Red Hat,2965.0,159.0,4594,188.0,3,Technology,1
3,0.0,Intel,1557.0,112.0,3504,190.0,4,Technology,1
4,0.0,IBM,1419.0,128.0,3515,259.0,5,Technology,1


In [18]:
df_sp = df_sp.rename(columns=
                         {'Name': 'company'
                             })
print(df_sp.shape)
df_sp['in_sp500']=1
df_sp.head(5)

(505, 3)


,Symbol,company,Sector,in_sp500
0,MMM,3M,Industrials,1
1,AOS,A. O. Smith,Industrials,1
2,ABT,Abbott Laboratories,Health Care,1
3,ABBV,AbbVie,Health Care,1
4,ABMD,Abiomed,Health Care,1


In [19]:
print(len(df_osci['company'].unique()))
print(len(df_sp['company'].unique()))

298
505


In [20]:
df_parta = df_sp.merge(df_osci, left_on="company", right_on="company", how="outer")
print(len(df_parta))
df_parta.head()

781


,Symbol,company,Sector,in_sp500,positionChange,activeContributors,activeContributorsChange,totalCommunity,totalCommunityChange,position,industry,in_osci
0,MMM,3M,Industrials,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOS,A. O. Smith,Industrials,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABT,Abbott Laboratories,Health Care,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBV,AbbVie,Health Care,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABMD,Abiomed,Health Care,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print(len(df_parta['company'].unique()))

781


In [22]:
df_parta = df_parta.rename(columns=
                         {'Symbol': 'symbol',
                          'Sector': 'sector',
                          'position': 'osci_position'
                             })

new_order = ['symbol', 'company','sector', 'industry',
             'in_sp500','in_osci', 'osci_position'
            ]

df_parta = df_parta.reindex(new_order, axis=1)

In [23]:
df_parta.head()

,symbol,company,sector,industry,in_sp500,in_osci,osci_position
0,MMM,3M,Industrials,NaN,1.0,NaN,NaN
1,AOS,A. O. Smith,Industrials,NaN,1.0,NaN,NaN
2,ABT,Abbott Laboratories,Health Care,NaN,1.0,NaN,NaN
3,ABBV,AbbVie,Health Care,NaN,1.0,NaN,NaN
4,ABMD,Abiomed,Health Care,NaN,1.0,NaN,NaN


In [24]:
df_check = df_parta.groupby(['in_sp500','in_osci']).count()
df_check

,,symbol,company,sector,industry,osci_position
in_sp500,in_osci,,,,,
1.0,1.0,22,22,22,22,22


In [25]:
df_check = df_parta.groupby(['in_sp500']).count()
df_check

,symbol,company,sector,industry,in_osci,osci_position
in_sp500,,,,,,
1.0,505,505,505,22,22,22


In [26]:
df_check = df_parta.groupby(['in_osci']).count()
df_check

,symbol,company,sector,industry,in_sp500,osci_position
in_osci,,,,,,
1.0,22,298,22,298,22,298


In [27]:
df_check = df_parta['company'].count() 
df_check

781

In [28]:
#df_parta.head(5)

In [29]:
#df_parta.dtypes

In [30]:
#df_sample = df_parta.sample(20)

In [31]:
#df_sample.sort_values(by=['company'])

In [32]:
df_parta[df_parta['company']=="Google"]

,symbol,company,sector,industry,in_sp500,in_osci,osci_position
505,NaN,Google,NaN,Technology,NaN,1.0,1.0


In [33]:
df_sorted = df_parta.sort_values(by=['company'])
df_sorted.head(5)

,symbol,company,sector,industry,in_sp500,in_osci,osci_position
0,MMM,3M,Industrials,NaN,1.0,NaN,NaN
767,NaN,4teamwork,NaN,Technology,NaN,1.0,285.0
758,NaN,5minds,NaN,Technology,NaN,1.0,276.0
1,AOS,A. O. Smith,Industrials,NaN,1.0,NaN,NaN
7,ADM,ADM,Consumer Staples,NaN,1.0,NaN,NaN


In [34]:
df_sample = df_sorted.sample(10)
df_sample

,symbol,company,sector,industry,in_sp500,in_osci,osci_position
727,NaN,Loodos Tech,NaN,Technology,NaN,1.0,244.0
357,PH,Parker-Hannifin,Industrials,NaN,1.0,NaN,NaN
306,MCK,McKesson Corporation,Health Care,NaN,1.0,NaN,NaN
554,NaN,Collabora,NaN,Technology,NaN,1.0,62.0
249,ICE,Intercontinental Exchange,Financials,NaN,1.0,NaN,NaN
241,IEX,IDEX Corporation,Industrials,NaN,1.0,NaN,NaN
315,MAA,Mid-America Apartments,Real Estate,NaN,1.0,NaN,NaN
313,MU,Micron Technology,Information Technology,NaN,1.0,NaN,NaN
548,NaN,WIX,NaN,Technology,NaN,1.0,56.0
620,NaN,HERE,NaN,Technology,NaN,1.0,137.0


### 3. Data Source:  Suzy spreadsheet

In [35]:
file_name = 'classification_suzy.csv'

df_suzy = pd.read_csv('data_raw/'+file_name)
df_suzy['in_suzy']=1

df_suzy = df_suzy.rename(columns=
                         {'Company': 'company',
                          'Level of Engagement': 'os_engagement',
                          'TODO Member': 'todo_member',
                          'TODO Adopter': 'todo_adopter',
                          'Main Business': 'area',
                          'Relevance to Core Business': 'relevance'
                             })

df_suzy.head(3)

,company,date_updated_suzy,Main Business,Area of Business Value,relevance,os_engagement,todo_member,todo_adopter,Linux Foundation (Platinum + Gold),GitHub list https://github.com/collections/open-source-organizations,OSCI Index top 60(as of Feb 2021),Reasoning,GitHub,Note,Response to survey,Survey: What do you consider to be your company's dominant open source investment where there are likely shared dependencies?,Survey: Want to hear more?,Emailed to:,in_suzy
0,AT&T,2021-05-07,Telcom/operator is main revenue source (also m...,software,Accelerant,Co-Leading,NaN,NaN,X,NaN,NaN,I sense that AT&T sees investments as strategi...,NaN,NaN,NaN,NaN,NaN,NaN,1
1,SAIC,2021-05-07,SAIC integrates multiple disciplines to provid...,services,Accelerant,Combining,NaN,NaN,NaN,NaN,NaN,SAIC gov't focused. They contribute (e.g. RedH...,https://github.com/saic-oss,NaN,NaN,NaN,NaN,NaN,1
2,Vapor IO,2021-05-07,edge computing infra,software,Accelerant,Co-Leading,NaN,NaN,NaN,NaN,NaN,"Using os for trust, function, ecosystem develo...",NaN,NaN,NaN,NaN,NaN,NaN,1


In [36]:
keep_cols = ['company', 'area','os_engagement', 'in_suzy', 'relevance' 
            ]
            #'activeContributors', 'activeContributorsChange', 
            # 'totalCommunity', 'totalCommunityChange', 'position', 'industry'
            #]

df_suzy = df_suzy.filter(keep_cols)

print(df_suzy.shape)
df_suzy.head(3)

(141, 4)


,company,os_engagement,in_suzy,relevance
0,AT&T,Co-Leading,1,Accelerant
1,SAIC,Combining,1,Accelerant
2,Vapor IO,Co-Leading,1,Accelerant


In [37]:
print(len(df_suzy['company'].unique()))

141


In [38]:
df_check = df_suzy.groupby(['in_suzy']).count()
df_check

,company,os_engagement,relevance
in_suzy,,,
1,141,55,55


In [39]:
df_partb = df_sorted.merge(df_suzy, left_on="company", right_on="company", how="outer")
df_partb['company_count']=1
print(df_partb.shape)
df_partb.head()

(876, 11)


,symbol,company,sector,industry,in_sp500,in_osci,osci_position,os_engagement,in_suzy,relevance,company_count
0,MMM,3M,Industrials,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
1,NaN,4teamwork,NaN,Technology,NaN,1.0,285.0,NaN,NaN,NaN,1
2,NaN,5minds,NaN,Technology,NaN,1.0,276.0,NaN,NaN,NaN,1
3,AOS,A. O. Smith,Industrials,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
4,ADM,ADM,Consumer Staples,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1


In [40]:
print(len(df_partb['company'].unique()))

876


In [41]:
df_check = df_partb.groupby(['in_osci','in_sp500','in_suzy','company_count']).count()
df_check

,,,,symbol,company,sector,industry,osci_position,os_engagement,relevance
in_osci,in_sp500,in_suzy,company_count,,,,,,,
1.0,1.0,1.0,1,8,8,8,8,8,3,3


### 4. Data Source:  OSPO Landscape(ToDo Group)
https://landscape.todogroup.org

In [42]:
file_name = 'ospo_todo_landscape.csv'

df_ospo = pd.read_csv('data_raw/'+file_name)
df_ospo['in_ospo']=1

print(df_ospo.shape)
df_ospo.head(3)

(225, 41)


,Name,Organization,Homepage,Logo,Twitter,Crunchbase URL,Market Cap,Ticker,Funding,Member,Relation,License,Headquarters,Latest Tweet Date,Description,Crunchbase Description,Crunchbase Homepage,Crunchbase City,Crunchbase Region,Crunchbase Country,Crunchbase Twitter,Crunchbase Linkedin,Crunchbase Ticker,Crunchbase Kind,Crunchbase Min Employees,Crunchbase Max Employees,Category,Subcategory,OSS,Github Repo,Github Stars,Github Description,Github Latest Commit Date,Github Latest Commit Link,Github Release Date,Github Release Link,Github Start Commit Date,Github Start Commit Link,Github Contributors Count,Github Contributors Link,in_ospo
0,Adobe (Member),Adobe,https://www.adobe.com/,https://landscape.todogroup.org/logos/adobe-me...,https://twitter.com/Adobe,https://www.crunchbase.com/organization/adobe,1.887401e+11,ADBE,2.500000e+06,General,member,NotOpenSource,"San Jose, California",2022-05-04 13:00:38,Adobe is a software company that provides its ...,Adobe is a software company that provides its ...,http://www.adobe.com,San Jose,California,United States,http://twitter.com/Adobe,https://www.linkedin.com/company/adobe,ADBE,market_cap,10001.0,1000000.0,TODO Group Member,General,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Aiven (Member),Aiven,https://github.com/aiven/,https://landscape.todogroup.org/logos/aiven-me...,https://twitter.com/aiven_io,https://www.crunchbase.com/organization/aiven,NaN,NaN,2.100000e+08,General,member,NotOpenSource,"Helsinki, Finland",2022-05-04 11:33:18,Aiven is an information technology company tha...,Aiven is an information technology company tha...,https://aiven.io,Helsinki,Southern Finland,Finland,https://twitter.com/aiven_io,https://www.linkedin.com/company/aiven/,NaN,funding,251.0,500.0,TODO Group Member,General,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Apple (Member),Apple,https://opensource.apple.com/,https://landscape.todogroup.org/logos/apple-me...,https://twitter.com/apple,https://www.crunchbase.com/organization/apple,2.608140e+12,AAPL,6.150230e+09,General,member,NotOpenSource,"Cupertino, California",NaN,Apple is a multinational corporation that desi...,Apple is a multinational corporation that desi...,http://www.apple.com,Cupertino,California,United States,https://twitter.com/apple,https://www.linkedin.com/company/apple,AAPL,market_cap,10001.0,1000000.0,TODO Group Member,General,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [43]:
df_ospo = df_ospo.rename(columns=
                         {'Organization': 'company',
                          'Ticker': 'symbol_ospo',
                          'Market Cap': 'market_cap',
                          'Crunchbase Country': 'country',
                          'License': 'license'
                             })

keep_cols = ['company', 'symbol_ospo','market_cap', 'country', 'relevance',
             'in_ospo'
            ]

df_ospo = df_ospo.filter(keep_cols)


print(df_ospo.shape)
df_ospo.head(3)

(225, 5)


,company,symbol_ospo,market_cap,country,in_ospo
0,Adobe,ADBE,1.887401e+11,United States,1
1,Aiven,NaN,NaN,Finland,1
2,Apple,AAPL,2.608140e+12,United States,1


In [44]:
#df_ospo

In [45]:
print(len(df_ospo['company'].unique()))

117


# NEXT STEPS
- rename vars in ospo dataframe
- merge ospo data in
- sort and check company names

In [46]:
df_check = df_ospo.groupby(['in_ospo']).count()
df_check

,company,symbol_ospo,market_cap,country
in_ospo,,,,
1,225,133,133,225


In [47]:
df_partc = df_partb.merge(df_ospo, left_on="company", right_on="company", how="outer")
df_partc['company_count']=1
print(df_partc.shape)
df_partc.head()

(1039, 15)


,symbol,company,sector,industry,in_sp500,in_osci,osci_position,os_engagement,in_suzy,relevance,company_count,symbol_ospo,market_cap,country,in_ospo
0,MMM,3M,Industrials,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
1,NaN,4teamwork,NaN,Technology,NaN,1.0,285.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
2,NaN,5minds,NaN,Technology,NaN,1.0,276.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
3,AOS,A. O. Smith,Industrials,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
4,ADM,ADM,Consumer Staples,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN


In [48]:
print(len(df_partc['company'].unique()))

931


In [49]:
df_check = df_partc.groupby(['in_sp500', 'in_osci', 'in_suzy', 'in_ospo','company_count']).count()
df_check

,,,,,symbol,company,sector,industry,osci_position,os_engagement,relevance,symbol_ospo,market_cap,country
in_sp500,in_osci,in_suzy,in_ospo,company_count,,,,,,,,,,
1.0,1.0,1.0,1.0,1,10,10,10,10,10,4,4,10,10,10


In [50]:
df_partc

,symbol,company,sector,industry,in_sp500,in_osci,osci_position,os_engagement,in_suzy,relevance,company_count,symbol_ospo,market_cap,country,in_ospo
0,MMM,3M,Industrials,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
1,NaN,4teamwork,NaN,Technology,NaN,1.0,285.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
2,NaN,5minds,NaN,Technology,NaN,1.0,276.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
3,AOS,A. O. Smith,Industrials,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
4,ADM,ADM,Consumer Staples,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
5,AES,AES Corp,Utilities,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
6,NaN,AMD,NaN,Technology,NaN,1.0,31.0,NaN,1.0,NaN,1,NaN,NaN,NaN,NaN
7,APA,APA Corporation,Energy,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
8,NaN,ARM,NaN,Technology,NaN,1.0,22.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
9,T,AT&T,Communication Services,NaN,1.0,NaN,NaN,Co-Leading,1.0,Accelerant,1,NaN,NaN,NaN,NaN


In [51]:
df_check = df_partc.groupby(['in_sp500']).count()
df_check

,symbol,company,sector,industry,in_osci,osci_position,os_engagement,in_suzy,relevance,company_count,symbol_ospo,market_cap,country,in_ospo
in_sp500,,,,,,,,,,,,,,
1.0,526,526,526,33,33,33,13,22,13,526,44,44,44,44


In [52]:
print("Today's date:", current_date)

Today's date: 2022-05-21


# END OF CODE

# END OF CODE

# END OF CODE

---

In [53]:
print x

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(x)? (3945978432.py, line 1)

### Lux video

#### Example hands-on Lux tutorial
https://mybinder.org/v2/gh/lux-org/lux-binder/master?urlpath=lab/tree/demo/hpi_covid_demo.ipynb


In [54]:
import pandas as pd
#df = pd.read_csv("data.csv")
import lux

df = df_partc.copy()
#df.groupby(" ").mean()["xxx"]

In [ ]:
df_meetup['city'].mask(df_meetup['city'] == 'Raleigh', 'Raleigh-Durham', inplace=True)



In [ ]:
# rename columns
df_meetup.rename(columns={"name": "meetup_name", 
                          "members": "meetup_members",
                          "fullurl": "meetup_url",
                          "created": "meetup_created",
                          "status": "meetup_status"
                            },
                   inplace = True,
                   errors="raise")

## 2. Data Source:  Chapter List
### Get raw data

In [ ]:
!ls -alt data_raw/

In [ ]:
file_name = '1_pyladies_export_2021_03_01.csv'

chapter_list = pd.read_csv(datapath+file_name)  

In [ ]:
chapter_list.head()

In [ ]:
chapter_list.info()

In [ ]:
chapter_list.shape

In [ ]:
df_chapter = chapter_list.copy()
# rename columns
df_chapter.rename(columns={"First Name [Required]": "city1", 
                             "Last Name [Required]": "city2", 
                             "Email Address [Required]": "email",
                             "Last Sign In [READ ONLY]": "email_signin_date",
                             "Email Usage [READ ONLY]": "email_usage"
                            },
                   inplace = True,
                   errors="raise")


# delete columns:  'active' is same for all
df_chapter = df_chapter.drop(['Status [READ ONLY]'], axis=1)

# fix data issue
df_chapter['city2'].mask(df_chapter['city2'] == \
                      'PyLladies', \
                      'PyLadies', inplace=True)

df_chapter['city2'].mask(df_chapter['city2'] == \
                      'PyLdies', \
                      'PyLadies', inplace=True)

# swap some rows
df_chapter['city_temp1'] = df_chapter['city1'].str.upper() 
df_chapter['city_temp2'] = df_chapter['city2'].str.upper() 

df_chapter['city'] = df_chapter['city1']
df_chapter['temp'] = df_chapter['city2']

df_chapter['city'].mask(df_chapter['city_temp1'] == 'PYLADIES', df_chapter['city2'], inplace=True)

df_chapter['temp'].mask(df_chapter['city_temp2'] == 'PYLADIES', df_chapter['city1'], inplace=True)

# exclude some rows
df_chapter = df_chapter[(df_chapter['city_temp1'] == 'PYLADIES') | (df_chapter['city_temp2'] == 'PYLADIES')] 

df_chapter = df_chapter[(df_chapter['city1'] != 'Admin')] 
df_chapter = df_chapter[(df_chapter['city2'] != 'Sponsorship')] 
df_chapter = df_chapter[(df_chapter['city2'] != 'Info')] 
df_chapter = df_chapter[(df_chapter['city2'] != 'Systems')] 
df_chapter = df_chapter[(df_chapter['city1'] != 'Communications')] 

# clean up city names
df_chapter['city'].mask(df_chapter['city'] == 'NYC', 'New York', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Princeton NJ', 'Princeton', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Pune, India', 'Pune', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'TC', 'Minneapolis', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Goa', 'Panaji Goa', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Visakhapatnam/Andhra Pradesh/India', 'Visakhapatnam', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Washington', 'Washington DC', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Aba / Abia State', 'Aba', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'West Windsor NJ', 'West Windsor', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Medellin', 'Medellín', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Albania', 'Tirania', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'BH', 'Belo Horizonte', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Guatemala', 'Guatemala City', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Inland Empire', 'Riverside', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Vale', 'Vale do Paraiba', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Plateau', 'Jos Plateau', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Recife, Brasil', 'Recife', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Rio de Janerio', 'Rio de Janeirio', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Salt Lake', 'Salt Lake City', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'SA', 'San Antonio', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'São Carlos, Brasil', 'São Carlos', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Taiwan', 'Taipei', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Milan', 'Milano', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Munich', 'München', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Rio de Janeirio', 'Rio de Janeiro', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Bangalore', 'Bengaluru', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'DC', 'Washington DC', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'DF', 'Brasilia', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Congo-Brazzaville', 'Brazzaville', inplace=True)


import datetime
df_chapter['date_logged'] = df_chapter['email_signin_date']

df_chapter['email_signin_date'].mask(df_chapter['email_signin_date'] == 'Never logged in', 'NaN', inplace=True)

df_chapter['email_signin_date'] = df_chapter['email_signin_date'].astype('datetime64[ns]').dt.date

# drop temp variables
df_chapter = df_chapter.drop(['city1','city2','city_temp1','city_temp2','temp'], axis=1)

df_chapter['registered_db'] = 1

In [ ]:
df_chapter.head(300)

In [ ]:
df_chapter.shape

In [ ]:
df_all = pd.merge(df_chapter, df_meetup, on=['city'],  how='outer')

In [ ]:
print('emails (all):      ', chapter_list.shape)
print('emails (chapters): ', df_chapter.shape)
print('-'*30)
print('meetup dashboard:  ', df_meetup.shape)
print('-'*30)
print('Merged:            ', df_all.shape)

In [ ]:
df_all.sort_values(by=['city'])

In [ ]:
df_all[['email','email_signin_date','email_usage','city','country','region', 'registered_db','meetup_db','meetup_status','last_event']]

In [ ]:
df_all = df_all.drop(['country','region'], axis=1)

## 3. Data Source:  Reshama's Inventory
### Get raw data

In [ ]:
file_name3 = '2021_PyLadies_Inventory - 2021-03.csv'
df_inventory = pd.read_csv(datapath+file_name3)  
df_inventory['inv_db'] = 1

In [ ]:
#df_inventory['city'].mask(df_inventory['city'] == 'Caxias Do Sul', 'Caxias do Sul', inplace=True)
#df_inventory['city'].mask(df_inventory['city'] == 'Duque De Caxias', 'Duque de Caxias', inplace=True)
#df_inventory['city'].mask(df_inventory['city'] == 'Salt Lake Area', 'Salt Lake City', inplace=True)


In [ ]:
df_inventory.head(5)

In [ ]:
df_inventory.shape

In [ ]:
df_use = pd.merge(df_all, df_inventory, on=['city'],  how='outer')

In [ ]:
df_use = df_use.sort_values(by=['city'])

df_use.head(4)

In [ ]:
# As a default, if meetup group exists, use that member count

#df_test=df_use.copy()

# df_test[['city', 'country','region', \
#         'registered_db','meetup_db','main_platform','meetup_members','members_followers']].head(10)
# df_test['members_followers'].mask(df_test['meetup_members'] == 'NaN', df_test['members_followers'], inplace=True)


df_use['members_followers']=df_use['meetup_members'].mask(pd.isnull, df_use['members_manual'])


df_use[['city', 'country','region', \
        'registered_db','meetup_db','main_platform','meetup_members','members_manual','members_followers']].head(100)


In [ ]:
df_use[['city', 'country','region', \
        'registered_db','meetup_db','main_platform','meetup_members','members_followers']]


In [ ]:
df_use.shape

In [ ]:
#df_use[['email','email_signin_date','email_usage','city','country_x','region_x', 'country_y','region_y', 'registered_db','meetup_db','status','last_event']]
df_use[['email','email_signin_date','city', 'country','region', \
        'registered_db','meetup_db','main_platform']]


In [ ]:
df_use['chapter_count'] = 1

In [ ]:
df_use['meetup_db'].unique()

In [ ]:
df_use['meetup_db'].value_counts(normalize=True)

In [ ]:
df_use['meetup_db'].value_counts(normalize=False)

In [ ]:
#df_use.head(3)

In [ ]:
df_use.shape

### Remove Duplicates

In [ ]:
# get names of indexes for which 

# Note:  There are 2 registered chapters for Bengaluru
index_names = df_use[ df_use['email'] == 'bangalore@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'sa@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'mum@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'newark@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'alb@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'tampa@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'tunisia@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

# Note:  There are 2 meetup groups with same name
index_names = df_use[ df_use['meetup_url'] == 'https://www.meetup.com/Pyladies-Bogota/'].index 
df_use.drop(index_names, inplace = True) 

# Note:  There are 2 meetup groups with same name
index_names = df_use[ df_use['meetup_url'] == 'https://www.meetup.com/PyLadies-PUC-Rio/'].index 
df_use.drop(index_names, inplace = True) 


df_use.shape

### Create `location` column

In [ ]:
df_use['location'] = df_use['city'].astype(str) + ' ' + df_use['country'].astype(str)
df_use['location2']= df_use['city'].astype(str) \
                        + ' ' \
                        + df_use['state-province'].astype(str) \
                        + ' ' \
                        + df_use['country'].astype(str)


subset = df_use[['location','location2', 'city', 'state-province','country','country_level','region', 'main_platform']]
subset.head(300)

In [ ]:
df_use.head()

## For Venn Diagram

In [ ]:
# subset = df_use[df_use["meetup"].isin(["1"])]
# subset

In [ ]:
df_use[['email','email_signin_date','city', 'country','region', \
        'registered_db','meetup_db','main_platform']]

In [ ]:
df_use_check=df_use[['meetup_db','registered_db','inv_db','chapter_count']]

In [ ]:
df_check = df_use_check.groupby(['meetup_db']).sum()

In [ ]:
df_check

In [ ]:
df_check = df_use_check.groupby(['registered_db']).sum()

In [ ]:
df_check

In [ ]:
df_check = df_use_check.groupby(['inv_db']).sum()

In [ ]:
df_check

In [ ]:
df_check = df_use.groupby(['meetup_db','registered_db','inv_db']).sum()
#df_check = df_use.groupby(['meetup_db','registered_db','inv_db']).count()
df_check

In [ ]:
df_use[['meetup_db','registered_db','inv_db','email','email_signin_date','city', 'state-province','country','region', 'main_platform']]

In [ ]:
df_use.shape

In [ ]:
# subset = df_use.query('meetup_db' in(1))
# #& registered_db == '1')


# temp = df_use[df_use.eval("meetup_db = '1'")]


In [ ]:
df_use['status_derived'].value_counts(normalize=False)

In [ ]:
df_use['main_platform'].value_counts(normalize=False)

In [ ]:
subset = df_use[df_use["country_level"].isin(["1"])]
print(subset.shape)
subset.head()


In [ ]:
df_use['region'].value_counts(normalize=False)

In [ ]:
df_use['members_followers'].sum()

In [ ]:
(df_use['country'].unique()).shape

In [ ]:
df_use['country'].value_counts(normalize=False)[:10]

In [ ]:
# 75 meetup
# 60 not found
# 135
# 261 - 135 = 125 -45 Brazil = 90



# Set cvs file

In [ ]:
#df_use.sort_values(by=['region','country','city','state-province'])

df_use.sort_values(['region','country','city','state-province'], inplace=True, ascending=True)


In [ ]:
df_use.to_csv('data_derived/pyladies_clean.csv',
              columns=['region','country','city','state-province','email','email_signin_date','email_usage', \
                       'main_platform','founded','meetup_created','main_platform_link','members_followers', \
                       'status_derived','members_manual','meetup_members', 'meetup_url', 'meetup_status','last_event',\
                       'past_events','upcoming_events','country_level','last_activity', 'facebook_page',\
                       'website','meetup_link', 'twitter_link','instagram_link','other_social_media', \
                       'meetup_db','registered_db','inv_db','meetup_name','location','location2','chapter_count'
                                 ])


In [ ]:
#df_use.info()

In [ ]:
df_use.head()